[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://github.com/milvus-io/bootcamp/blob/master/bootcamp/tutorials/quickstart/hybrid_serach_with_milvus.ipynb)

# Hybrid Search with Dense and Sparse Vectors in Milvus

In this tutorial, we will demonstrate how to conduct hybrid search with [Milvus](https://milvus.io/docs/multi-vector-search.md) and [BGE-M3 model](https://github.com/FlagOpen/FlagEmbedding/tree/master/FlagEmbedding/BGE_M3). BGE-M3 model can convert text into dense and sparse vectors. Milvus supports storing both types of vectors in one collection, allowing for hybrid search that enhances the result relevance.

Milvus supports Dense, Sparse, and Hybrid retrieval methods:

- Dense Retrieval: Utilizes semantic context to understand the meaning behind queries.
- Sparse Retrieval: Emphasizes keyword matching to find results based on specific terms, equivalent to full-text search.
- Hybrid Retrieval: Combines both Dense and Sparse approaches, capturing the full context and specific keywords for comprehensive search results.

By integrating these methods, the Milvus Hybrid Search balances semantic and lexical similarities, improving the overall relevance of search outcomes. This notebook will walk through the process of setting up and using these retrieval strategies, highlighting their effectiveness in various search scenarios.

### Dependencies and Environment

In [ ]:
!pip install --upgrade pymilvus "pymilvus[model]"

### Download Dataset

To demonstrate search, we need a corpus of documents. Let's use the Quora Duplicate Questions dataset and place it in the local directory. 

Source of the dataset: [First Quora Dataset Release: Question Pairs](https://www.quora.com/q/quoradata/First-Quora-Dataset-Release-Question-Pairs)

In [ ]:
# Run this cell to download the dataset
!wget http://qim.fs.quoracdn.net/quora_duplicate_questions.tsv

### Load and Prepare Data

We will load the dataset and prepare a small corpus for search.

In [1]:
import pandas as pd

file_path = "quora_duplicate_questions.tsv"
df = pd.read_csv(file_path, sep="\t")
questions = set()
for _, row in df.iterrows():
    obj = row.to_dict()
    questions.add(obj["question1"][:512])
    questions.add(obj["question2"][:512])
    if len(questions) > 10000:
        break

docs = list(questions)

# example question
print(docs[0])

What happened to Luna Lovegood after Book 7?


### Use BGE-M3 Model for Embeddings

The BGE-M3 model can embed texts as dense and sparse vectors. 

In [2]:
from milvus_model.hybrid import BGEM3EmbeddingFunction

ef = BGEM3EmbeddingFunction(use_fp16=False, device="cpu")
dense_dim = ef.dim["dense"]

# Generate embeddings using BGE-M3 model
docs_embeddings = ef(docs)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Inference Embeddings: 100%|██████████| 626/626 [04:39<00:00,  2.24it/s]


### Generate Random Embeddings (Optional)

If you do not have the BGE-M3 model, you can generate random embeddings for demonstration purposes.

In [ ]:
import random
import numpy as np


def random_embedding(texts):
    rng = np.random.default_rng()
    return {
        "dense": np.random.rand(len(texts), 768),
        "sparse": [
            {
                d: rng.random()
                for d in random.sample(range(1000), random.randint(20, 30))
            }
            for _ in texts
        ],
    }


dense_dim = 768
ef = random_embedding
docs_embeddings = ef(docs)

### Setup Milvus Collection and Index

We will now set up the Milvus collection and create indices for the vector fields.

In [ ]:
from pymilvus import (
    FieldSchema,
    CollectionSchema,
    DataType,
    Collection,
    connections,
)

# Connect to Milvus
connections.connect("default", uri="milvus.db")

# Specify the data schema for the new Collection
fields = [
    # Use auto generated id as primary key
    FieldSchema(
        name="pk", dtype=DataType.VARCHAR, is_primary=True, auto_id=True, max_length=100
    ),
    # Store the original text to retrieve based on semantically distance
    FieldSchema(name="text", dtype=DataType.VARCHAR, max_length=512),
    # Milvus now supports both sparse and dense vectors,
    # we can store each in a separate field to conduct hybrid search on both vectors
    FieldSchema(name="sparse_vector", dtype=DataType.SPARSE_FLOAT_VECTOR),
    FieldSchema(name="dense_vector", dtype=DataType.FLOAT_VECTOR, dim=dense_dim),
]
schema = CollectionSchema(fields, "")
col_name = "hybrid_demo"
col = Collection(col_name, schema, consistency_level="Strong")

# To make vector search efficient, we need to create indices for the vector fields
sparse_index = {"index_type": "SPARSE_INVERTED_INDEX", "metric_type": "IP"}
col.create_index("sparse_vector", sparse_index)
dense_index = {"index_type": "FLAT", "metric_type": "IP"}
col.create_index("dense_vector", dense_index)
col.load()

In [29]:
from pymilvus import MilvusClient, DataType

# Connect to Milvus
client = MilvusClient(uri="./milvus_demo.db")

# Specify the data schema for the new Collection
schema = MilvusClient.create_schema(
    auto_id=True,  # Auto-generated id
    enable_dynamic_field=True,
)

# Add fields to the schemax
schema.add_field(
    field_name="pk", datatype=DataType.VARCHAR, is_primary=True, max_length=100
)
schema.add_field(field_name="text", datatype=DataType.VARCHAR, max_length=512)
schema.add_field(field_name="sparse_vector", datatype=DataType.SPARSE_FLOAT_VECTOR)
schema.add_field(
    field_name="dense_vector", datatype=DataType.FLOAT_VECTOR, dim=128
)  # assuming dense_dim is 128

# Create the collection with the specified schema
collection_name = "hybrid_demo"
client.create_collection(collection_name=collection_name, schema=schema)

index_params = MilvusClient.prepare_index_params()

# Create index for the sparse vector
index_params.add_index(
    field_name="sparse_vector",
    metric_type="IP",
    index_type="SPARSE_INVERTED_INDEX",
)

# Create index for the dense vector
index_params.add_index(
    field_name="dense_vector",
    metric_type="IP",
    index_type="FLAT",
)

client.create_index(
    collection_name=collection_name,
    index_params=index_params,
)

# Load the collection
client.load_collection(collection_name=collection_name)

### Insert Data into Milvus Collection
Insert the text and sparse/dense vector representations into the collection.

In [ ]:
# Insert documents and their embeddings into the collection.
# For efficiency, we insert 50 records in each small batch
entities = [docs, docs_embeddings["sparse"], docs_embeddings["dense"]]
for i in range(0, len(docs), 50):
    batched_entities = [
        docs[i : i + 50],
        docs_embeddings["sparse"][i : i + 50],
        docs_embeddings["dense"][i : i + 50],
    ]
    col.insert(batched_entities)
col.flush()

### Enter Your Search Query

In [ ]:
# Enter your search query
query = input("Enter your search query: ")
print(query)

# Generate embeddings for the query
query_embeddings = ef([query])
print(query_embeddings)

Who started AI research?
{'dense': [array([-0.03658685, -0.01750261, -0.01536112, ..., -0.02266536,
        0.01365146,  0.00908284], dtype=float32)], 'sparse': <1x250002 sparse array of type '<class 'numpy.float32'>'
	with 5 stored elements in Compressed Sparse Row format>}


### Run the Search

In [ ]:
from pymilvus import (
    AnnSearchRequest,
    WeightedRanker,
)


def get_collection():
    col_name = "hybrid_demo"
    connections.connect("default", uri="milvus.db")
    col = Collection(col_name)
    return col


def hybrid_search(query_embeddings, sparse_weight=1.0, dense_weight=1.0):
    col = get_collection()
    sparse_search_params = {"metric_type": "IP"}
    sparse_req = AnnSearchRequest(
        query_embeddings["sparse"], "sparse_vector", sparse_search_params, limit=10
    )
    dense_search_params = {"metric_type": "IP"}
    dense_req = AnnSearchRequest(
        query_embeddings["dense"], "dense_vector", dense_search_params, limit=10
    )
    rerank = WeightedRanker(sparse_weight, dense_weight)
    res = col.hybrid_search(
        [sparse_req, dense_req], rerank=rerank, limit=10, output_fields=["text"]
    )
    if len(res):
        return [hit.fields["text"] for hit in res[0]]
    else:
        return []

In [ ]:
dense_results = hybrid_search(query_embeddings, sparse_weight=0.0, dense_weight=1.0)
sparse_results = hybrid_search(query_embeddings, sparse_weight=1.0, dense_weight=0.0)
hybrid_results = hybrid_search(query_embeddings, sparse_weight=0.7, dense_weight=1.0)

### Display Search Results

Display the results for Dense, Sparse, and Hybrid methods.

In [ ]:
def get_tokenizer():
    tokenizer = ef.model.tokenizer
    return tokenizer


def doc_text_formatting(query, docs):
    tokenizer = get_tokenizer()
    query_tokens_ids = tokenizer.encode(query, return_offsets_mapping=True)
    query_tokens = tokenizer.convert_ids_to_tokens(query_tokens_ids)
    formatted_texts = []

    for doc in docs:
        ldx = 0
        landmarks = []
        encoding = tokenizer.encode_plus(doc, return_offsets_mapping=True)
        tokens = tokenizer.convert_ids_to_tokens(encoding["input_ids"])[1:-1]
        offsets = encoding["offset_mapping"][1:-1]
        for token, (start, end) in zip(tokens, offsets):
            if token in query_tokens:
                if len(landmarks) != 0 and start == landmarks[-1]:
                    landmarks[-1] = end
                else:
                    landmarks.append(start)
                    landmarks.append(end)
        close = False
        formatted_text = ""
        for i, c in enumerate(doc):
            if ldx == len(landmarks):
                pass
            elif i == landmarks[ldx]:
                if close is True:
                    formatted_text += "]"
                else:
                    formatted_text += "["
                close = not close
                ldx = ldx + 1
            formatted_text += c
        if close is True:
            formatted_text += "]"
        formatted_texts.append(formatted_text)
    return formatted_texts

In [ ]:
# Dense search results
print("Dense Search Results:")
formatted_results = doc_text_formatting(query, dense_results)
for result in dense_results:
    print(result)

# Sparse search results
print("\nSparse Search Results:")
formatted_results = doc_text_formatting(query, sparse_results)
for result in formatted_results:
    print(result)

# Hybrid search results
print("\nHybrid Search Results:")
formatted_results = doc_text_formatting(query, hybrid_results)
for result in formatted_results:
    print(result)

Dense Search Results:
What's the best way to start learning robotics?
When in history did we start giving people names?
Why did humans come into existence?
Who invented thermometer?
Should a machine learning beginner go straight for deep learning?
How do I start learning or strengthen my knowledge of data structures and algorithms?
How can undergraduate help with machine learning research?
Why do so many people believe that the IQ test determines your intelligence?
Why is the term "research" used instead of scientific investigation?
Do humans fear artificial intelligence because it has no soul?
How do I research for MUN?
What is subjectivity and objectivity in research?
What are the top research fields in electrical engineering?
What is the best way to do an MUN research?
What advice will you give to an IIT graduate in Mechanical/Civil Engineering, who strongly wants to pursue a research career in computer science/mathematics, and has completed the basic courses of CS in coursera?
What

### Quick Deploy

To learn about how to start an online demo with this tutorial, please refer to [the example application](https://github.com/milvus-io/bootcamp/tree/master/bootcamp/tutorials/quickstart/apps/hybrid_demo_with_milvus).

<img src="https://raw.githubusercontent.com/milvus-io/bootcamp/master/bootcamp/tutorials/quickstart/apps/hybrid_demo_with_milvus/pics/demo.png"/>